# Import & Basic Setup

In [15]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [16]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, callbacks

print("TensorFlow:", tf.__version__)

TensorFlow: 2.19.0


# Paths & global config

In [17]:
# === EDIT THESE ===
WINDOW_ROOT = "/home/tonyliao/Location_new_aoa_PDF"   # folder containing amp_window_XXXXX.npy
ENC_SAVE_PATH = "/home/tonyliao/Location_new_aoa_PDF/models/prec_encoder.h5"
CLS_SAVE_PATH = "/home/tonyliao/Location_new_aoa_PDF/models/prec_classifier.h5"

# presence labels: 0 = empty, 1 = non-empty
CLASS_NAMES = ["Door_PreCNN", "Left_PreCNN", "Right_PreCNN","Top_PreCNN", "Empty_PreCNN","Middle_PreCNN","Above_PreCNN","RuJun_PreCNN","ChenZhu_PreCNN","Below_PreCNN"]

# training config
BATCH_SIZE = 64
EPOCHS = 40
LR = 1e-3
VAL_SPLIT = 0.2
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)

# PreCNN feature helper

## PreCNN Feature Construction

The amplitude window is:

$$
H(t,s,a) \in \mathbb{R}^{T \times S \times A}
$$

Averaging across antennas:

$$
\tilde{H}(t,s) = \frac{1}{A} \sum_{a=1}^{A} H(t,s,a)
$$

Transpose to channel-first:

$$
X_{\text{amp}} = \tilde{H}^{\top} \in \mathbb{R}^{S \times T}
$$

Global normalization:

$$
X_{\text{norm}}(c,t) = \frac{X_{\text{amp}}(c,t) - \mu_c}{\sigma_c + 10^{-6}}
$$

Final PreCNN input:

$$
X_{\text{pre}}(t) = 
\left[
X_{\text{norm}}(:,t),\; \text{mean}_c,\; \text{std}_c
\right]
$$

In [18]:

USE_PRECNN_STATS = True  # Step-2 requirement: amp + temporal mean + temporal std

def add_precnn_stats_channels(amp_ct):
    """
    amp_ct: [C_base, T]
    returns: [C_pre, T]
    """
    C_base, T = amp_ct.shape

    mu = amp_ct.mean(axis=1, keepdims=True)         # [C_base,1]
    sd = amp_ct.std(axis=1, keepdims=True) + 1e-6   # [C_base,1]

    mu_rep = np.repeat(mu, T, axis=1)
    sd_rep = np.repeat(sd, T, axis=1)

    return np.concatenate([amp_ct, mu_rep, sd_rep], axis=0).astype(np.float32)

# Loader for MATLAB amp_window_*.npy

In [19]:
def load_precnn_input(path, mu=None, sigma=None):
    """
    path: amp_window_XXXXX.npy
    raw shape in file: [T, S, A]

    Output: X_pre [T, C_pre] ready for PreCNN.
    """
    amp = np.load(path).astype(np.float32)   # [T, S, A]

    # Step 1: average over antennas
    avecsi_like = amp.mean(axis=2)           # [T, S]

    # Step 2: convert to channels-first
    amp_ct = avecsi_like.T                   # [S, T]
    amp_ct = amp_ct - np.median(amp_ct, axis=1, keepdims=True)
    # Eq(6),(8): reduce distance/wall scale sensitivity
    q75 = np.percentile(amp_ct, 75, axis=1, keepdims=True)
    q25 = np.percentile(amp_ct, 25, axis=1, keepdims=True)
    amp_ct = amp_ct / (q75 - q25 + 1e-6)
    # Step 3: normalize per-channel (computed later)
    if mu is not None and sigma is not None:
        amp_ct = (amp_ct - mu[:, None]) / (sigma[:, None] + 1e-6)

    # Step 4: add PreCNN stat channels
    if USE_PRECNN_STATS:
        amp_ct = add_precnn_stats_channels(amp_ct)   # [C_pre, T]

    # Step 5: final format for Keras
    X_pre = np.transpose(amp_ct, (1, 0))             # [T, C_pre]
    return X_pre

# File listing + label function

In [20]:
def list_amp_files(root):
    return sorted(glob.glob(os.path.join(root, "**", "amp_window_*.npy"), recursive=True))

def infer_label(path):
    up = path.upper()
    if "EMPTY" in up:
        return 0
    return 1

amp_files = list_amp_files(WINDOW_ROOT)
print("Found windows:", len(amp_files))

Found windows: 404573


# Compute global channel statistics (mu, sigma)

### Global Channel Statistics (μ, σ)

For each subcarrier channel \(c\):

Mean:

$$
\mu_c 
= \frac{1}{N} \sum_{i=1}^{N}
\left(
\frac{1}{T} \sum_{t=1}^{T} X_i(c,t)
\right)
$$

Second moment:

$$
m_{2,c} =
\frac{1}{N} \sum_{i=1}^{N}
\left(
\frac{1}{T} \sum_{t=1}^{T} X_i(c,t)^2
\right)
$$

Standard deviation:

$$
\sigma_c = \sqrt{m_{2,c} - \mu_c^2}
$$

In [21]:
def compute_mu_sigma(paths, sample_limit=None):
    sums = None
    sq_sums = None
    count = 0

    for i, p in enumerate(paths):
        if sample_limit and i >= sample_limit:
            break

        raw = np.load(p)        # [T, S, A]
        avecsi_like = raw.mean(axis=2)   # [T, S]
        amp_ct = avecsi_like.T           # [S, T]
        amp_ct = amp_ct - np.median(amp_ct, axis=1, keepdims=True)
        # Eq(6),(8): reduce distance/wall scale sensitivity
        q75 = np.percentile(amp_ct, 75, axis=1, keepdims=True)
        q25 = np.percentile(amp_ct, 25, axis=1, keepdims=True)
        amp_ct = amp_ct / (q75 - q25 + 1e-6)    
        C, _ = amp_ct.shape
        if sums is None:
            sums = np.zeros((C,), dtype=np.float64)
            sq_sums = np.zeros((C,), dtype=np.float64)

        sums    += amp_ct.mean(axis=1)
        sq_sums += (amp_ct**2).mean(axis=1)
        count += 1

    mu = sums / count
    sigma = np.sqrt(sq_sums / count - mu**2)
    return mu.astype(np.float32), sigma.astype(np.float32)

mu_global, sigma_global = compute_mu_sigma(amp_files)
print("mu:", mu_global.shape, "sigma:", sigma_global.shape)

mu: (53,) sigma: (53,)


# Build dataset (X, y)

In [22]:
X_list = []
y_list = []

for path in amp_files:
    x = load_precnn_input(path, mu=mu_global, sigma=sigma_global)
    X_list.append(x)
    y_list.append(infer_label(path))

X = np.stack(X_list, axis=0)    # [N, T, C_pre]
y = np.array(y_list)

print("X shape:", X.shape, "y shape:", y.shape)

X shape: (404573, 4, 159) y shape: (404573,)


# Train/val split

In [23]:
N = X.shape[0]
idx = np.arange(N)
np.random.shuffle(idx)

split = int(N * (1 - VAL_SPLIT))
train_idx = idx[:split]
val_idx   = idx[split:]

X_train, y_train = X[train_idx], y[train_idx]
X_val,   y_val   = X[val_idx],   y[val_idx]

print("Train:", X_train.shape, "Val:", X_val.shape)

Train: (323658, 4, 159) Val: (80915, 4, 159)


# Build PreCNN backbone + classifier

# PreCNN Backbone (Encoder)

The encoder applies temporal convolutions:

$$
X^{(1)} = \text{Conv1D}_{64, k=5}(X_{\text{pre}})
$$

$$
X^{(2)} = \text{Conv1D}_{128, k=5}(\text{Pool}(X^{(1)}))
$$

$$
X^{(3)} = \text{Conv1D}_{256, k=3}(\text{Pool}(X^{(2)}))
$$

Global average pooling:

$$
f = \text{GAP}(X^{(3)})
$$

Embedding vector:

$$
z = \text{Dense}_{128}(f)
$$


### Softmax Classifier

$$
\hat{y}_k = 
\frac{e^{w_k^{\top} z}}
{\sum_{j=1}^{K} e^{w_j^{\top} z}}
$$

In [24]:
def build_precnn_backbone(T, C):
    inp = layers.Input(shape=(T, C))

    x = layers.Conv1D(64, 5, padding="same", activation="relu")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)

    x = layers.Conv1D(128, 5, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(2)(x)

    x = layers.Conv1D(256, 3, padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)

    emb = layers.Dense(128, activation="relu", name="precnn_embedding")(x)
    return models.Model(inp, emb, name="PreCNN_Encoder")


def build_classifier(encoder, num_classes):
    inp = encoder.input
    feat = encoder.output
    x = layers.Dropout(0.3)(feat)
    out = layers.Dense(num_classes, activation="softmax")(x)
    return models.Model(inp, out, name="PreCNN_Classifier")


T, C_pre = X_train.shape[1], X_train.shape[2]
encoder = build_precnn_backbone(T, C_pre)
clf = build_classifier(encoder, len(CLASS_NAMES))

clf.summary()

I0000 00:00:1768479752.642814  907074 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22181 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "PreCNN_Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4, 159)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 4, 64)          │        50,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 2, 128)         │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 2, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 1, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ precnn_embedding (Dense)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 226,570 (885.04 KB)

 Trainable params: 225,674 (881.54 KB)

 Non-trainable params: 896 (3.50 KB)

# Train classifier

### Loss Function

Training minimizes sparse categorical cross-entropy:

$$
\mathcal{L}
=
- \log \left( \hat{y}_{\,y_{\text{true}}} \right)
$$

Adam optimizer updates parameters:

$$
\theta \leftarrow \theta - \eta \, \frac{\partial \mathcal{L}}{\partial \theta}
$$

In [25]:
clf.compile(
    optimizer=optimizers.Adam(LR),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

ckpt = callbacks.ModelCheckpoint(
    CLS_SAVE_PATH, save_best_only=True, monitor="val_accuracy", verbose=1
)
es = callbacks.EarlyStopping(
    monitor="val_accuracy", patience=6, restore_best_weights=True, verbose=1
)

clf.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[ckpt, es],
    verbose=2
)

print("Classifier saved:", CLS_SAVE_PATH)

Epoch 1/40


I0000 00:00:1768479754.173182  907240 service.cc:152] XLA service 0x7fbf8c024a10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1768479754.173196  907240 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2026-01-15 12:22:34.195297: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1768479754.319249  907240 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1768479755.881006  907240 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2026-01-15 12:22:46.714463: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_146', 4 bytes spill stores, 4 bytes spill loads

2026-01-15 12:22:47.943840: I external/l


Epoch 1: val_accuracy improved from -inf to 0.91103, saving model to /home/tonyliao/Location_new_aoa_PDF/models/prec_classifier.h5


5058/5058 - 15s - 3ms/step - accuracy: 0.9067 - loss: 0.3140 - val_accuracy: 0.9110 - val_loss: 0.2964
Epoch 2/40

Epoch 2: val_accuracy did not improve from 0.91103
5058/5058 - 10s - 2ms/step - accuracy: 0.9089 - loss: 0.2939 - val_accuracy: 0.9105 - val_loss: 0.2888
Epoch 3/40

Epoch 3: val_accuracy did not improve from 0.91103
5058/5058 - 12s - 2ms/step - accuracy: 0.9088 - loss: 0.2884 - val_accuracy: 0.9109 - val_loss: 0.2864
Epoch 4/40

Epoch 4: val_accuracy did not improve from 0.91103
5058/5058 - 10s - 2ms/step - accuracy: 0.9088 - loss: 0.2854 - val_accuracy: 0.9110 - val_loss: 0.2895
Epoch 5/40

Epoch 5: val_accuracy did not improve from 0.91103
5058/5058 - 10s - 2ms/step - accuracy: 0.9088 - loss: 0.2830 - val_accuracy: 0.9109 - val_loss: 0.2865
Epoch 6/40

Epoch 6: val_accuracy did not improve from 0.91103
5058/5058 - 11s - 2ms/step - accuracy: 0.9087 - loss: 0.2814 - val_accuracy: 0.9107 - val_loss: 0.2867
Epoch 7/40

Epoch 7: val_accuracy did not improve from 0.91103
5058

# Save encoder alone (for Step 3 & Step 4)

In [26]:
encoder.save(ENC_SAVE_PATH)
print("Encoder saved:", ENC_SAVE_PATH)
#Release resources
tf.keras.backend.clear_session()

Encoder saved: /home/tonyliao/Location_new_aoa_PDF/models/prec_encoder.h5


# Reset Kernel

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()
#Restart the kernel to free memory
import IPython
app = IPython.get_ipython()
app.kernel.do_shutdown(True)  # True = restart, False = shutdown

{'status': 'ok', 'restart': True}

: 